In [6]:
import pandas as pd
import torch
from transformers import PatchTSTForPrediction, PatchTSTConfig
from sklearn.preprocessing import StandardScaler
import numpy as np

# --- Opciones de visualización de pandas (para una salida más limpia) ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


### # 1. Cargar el dataset de ejemplo

In [7]:


try:
    df = pd.read_csv('clima_prueba.csv')
    print("--- Dataset cargado ---")
    print(df.head())
except FileNotFoundError:
    print("Error: El archivo 'clima_prueba.csv' no se encontró. Asegúrate de que esté en la misma carpeta.")
    exit()

--- Dataset cargado ---
                 Fecha  Temperatura  Humedad  Velocidad del Viento  Precipitacion
0  2025-01-01 00:00:00        28.50       85                  5.20           0.00
1  2025-01-01 01:00:00        28.10       86                  6.00           0.00
2  2025-01-01 02:00:00        27.80       87                  5.50           0.00
3  2025-01-01 03:00:00        27.50       88                  5.80           0.00
4  2025-01-01 04:00:00        27.20       89                  6.10           0.00


### 2. Preprocesar y normalizar los datos

In [8]:


time_index = pd.to_datetime(df['Fecha'])
data_values = df[['Temperatura', 'Humedad', 'Velocidad del Viento', 'Precipitacion']].values.astype(np.float32)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_values)

### 3. Dividir el dataset para entrenamiento e inferencia

In [9]:

past_values = scaled_data[:20]
future_values = scaled_data[20:]

past_values_tensor = torch.tensor(past_values).unsqueeze(0)
future_values_tensor = torch.tensor(future_values).unsqueeze(0)

### 4. Cargar el modelo PatchTST

In [10]:

config = PatchTSTConfig(
    num_input_channels=4,
    prediction_length=4,
    context_length=20,
    patch_length=4,
    patch_stride=2,
    num_time_features=0,
    output_range=None,
    normalization_config=None
)
model = PatchTSTForPrediction(config)


### 5. Entrenar (o realizar un 'forward pass' de ejemplo)

In [11]:

print("\n--- Paso de entrenamiento (con 'future_values') ---")
outputs_train = model(
    past_values=past_values_tensor,
    future_values=future_values_tensor
)
loss = outputs_train.loss
print(f"Pérdida de entrenamiento: {loss.item():.4f}")


--- Paso de entrenamiento (con 'future_values') ---
Pérdida de entrenamiento: 0.4797


### 6. Realizar una predicción ('inferencia')

In [12]:

print("\n--- Paso de inferencia (solo con 'past_values') ---")
outputs_inference = model(
    past_values=past_values_tensor
)
prediction_outputs = outputs_inference.prediction_outputs[0].detach().numpy()
predicted_values = scaler.inverse_transform(prediction_outputs)



--- Paso de inferencia (solo con 'past_values') ---


### 7. Visualización mejorada de los resultados

In [13]:

columnas = ['Temperatura', 'Humedad', 'Velocidad del Viento', 'Precipitacion']
df_reales = df.iloc[20:].reset_index(drop=True)

df_predicciones = pd.DataFrame(data=predicted_values, columns=columnas)
last_date = pd.to_datetime(df['Fecha'].iloc[-1])
start_date_pred = last_date + pd.Timedelta(hours=1)
fechas_prediccion = pd.date_range(start=start_date_pred, periods=4, freq='h') # Corregido a 'h'

df_predicciones['Fecha'] = fechas_prediccion
df_predicciones = df_predicciones[['Fecha'] + columnas]

# Juntar y mostrar los DataFrames para una fácil comparación
print("\n--- Comparación de Valores Reales vs. Predichos ---")

comparacion = pd.concat([df_reales, df_predicciones], axis=1, keys=['Reales', 'Predichos'])
print(comparacion)

print("\nPredicciones del modelo mostradas arriba, con valores reales a la izquierda y predichos a la derecha.")
print("La diferencia entre ambas es la precisión del modelo en este ejemplo de un solo paso.")


--- Comparación de Valores Reales vs. Predichos ---
                Reales                                                                  Predichos                                                       
                 Fecha Temperatura Humedad Velocidad del Viento Precipitacion               Fecha Temperatura Humedad Velocidad del Viento Precipitacion
0  2025-01-01 20:00:00       30.50      75                 7.50          0.00 2025-01-02 00:00:00       31.94   72.42                 9.04          0.00
1  2025-01-01 21:00:00       29.20      80                 6.20          0.00 2025-01-02 01:00:00       31.67   71.84                 8.83         -0.00
2  2025-01-01 22:00:00       28.80      82                 5.50          0.00 2025-01-02 02:00:00       31.79   71.95                 8.93          0.00
3  2025-01-01 23:00:00       28.60      84                 5.00          0.00 2025-01-02 03:00:00       31.75   71.70                 8.89          0.00

Predicciones del modelo most